# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [1]:
import sys; sys.path.insert(0, '..')
import itertools

In [2]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [3]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [4]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [5]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [6]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [7]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [8]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [9]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [10]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [11]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [12]:
main_node = "beg"
delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [17]:
query = '[lemma="beg"]'

In [18]:
result = topic_concordancer.main(query,window=50,topicn=25)

Training lda model began
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5Blemma%3D%22beg%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=50
training of gensim corpus began
gensim corpus done


### Print the key topics

In [19]:
for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

0
0.194*"do_not" + 0.136*"walk" + 0.129*"camp" + 0.116*"find" + 0.114*"eat" + 0.025*"go_through" + 0.022*"every_day" + 0.019*"body" + 0.017*"Americans" + 0.015*"strength"


1
0.142*"pardon_?" + 0.111*"send" + 0.059*"MB" + 0.055*"write" + 0.042*"letter" + 0.041*"kind_of" + 0.040*"thing" + 0.034*"America" + 0.034*"Beg" + 0.028*"answer"


2
0.137*"back" + 0.089*"go_back" + 0.088*"come_back" + 0.082*"do_not" + 0.061*"able_to" + 0.058*"school" + 0.057*"little_bit" + 0.049*"let_'s" + 0.029*"order" + 0.027*"end"


3
0.168*"stay" + 0.148*"day" + 0.112*"place" + 0.068*"night" + 0.059*"finally" + 0.046*"move" + 0.041*"doctor" + 0.040*"sleep" + 0.028*"winter" + 0.027*"russian"


4
0.136*"jewish" + 0.090*"home" + 0.059*"afraid" + 0.058*"speak" + 0.058*"village" + 0.055*"polish" + 0.047*"boy" + 0.035*"decide" + 0.035*"good" + 0.035*"job"


5
0.266*"food" + 0.144*"house" + 0.048*"farmer" + 0.044*"potato" + 0.040*"group" + 0.032*"at_night" + 0.028*"car" + 0.023*"wagon" + 0.023*"horse" + 0.021*"most_o

### Analyze documents

In [20]:
i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

So what did you do ? They opened displaced persons camps , and they flooded them with food from America , from England , you know ? And people , so many people , died from overeating . They got sick . They could n't take it . And I begged them , please , eat slowly . You are like a baby . First , the mother milk , and then you get the cereal . You did n't eat for four , five years this kind of food . You will get sick . Eat slowly . You do 


second day at night , they were looking over the -- the hiding places . They found many people , children . We heard children 's screams . Mothers -- do n't hit her . Please . Why are you doing it to us ? Do n't shoot us , begging . And then on the third night , it 's quieted down . Just the smell of burning bodies . So I walked out . I could hardly walk after sitting down for -- for two -- two days -- three days like this . INTERVIEWER : Did you think 


walking . It took 28 arduous days to get to my hometown . INT : And how did you survive durin

## Testimonial fragments

### 1.  

In [19]:
lemmas = ["beg","child"]

In [20]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="beg"][]{0,25}[lemma="child"])|([lemma="child"][]{0,25}[lemma="beg"])


In [21]:
domain_term = "for the child"

In [22]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [23]:
fragment_1 = {}
fragment_1['original_sentence'] = "He, he begged them to let to let the children that somebody should come and pick up the children."
fragment_1['label']="He, he begged them to let to let the children that somebody should come and pick up the children."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22He%22%5D+%5B%5D+%5B%22he%22%5D+%5B%22begged%22%5D+%5B%22them%22%5D+%5B%22to%22%5D+%5B%22let%22%5D+%5B%22to%22%5D+%5B%22let%22%5D+%5B%22the%22%5D+%5B%22children%22%5D+%5B%22that%22%5D+%5B%22somebody%22%5D+%5B%22should%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22pick%22%5D+%5B%22up%22%5D+%5B%22the%22%5D+%5B%22children%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'He , he begged them to let to let the children that somebody should come and pick up the children . ', 'right': '', 'complete_match': 'He , he begged them to let to let the children that somebody should come and pick up the children . ', 'testimony_id': 'irn504601', 'shelfmark': ['USHMM RG-50.030*0107'], 'token_start': 3956, 'token_end': 3977}


In [24]:
fragment_2 = {}
fragment_2['original_sentence'] = "some of the mothers were begging the Nazis to allow them to escort their children."
fragment_2['label']="(..) some of the mothers were begging the Nazis to allow them to escort their children."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22some%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22mothers%22%5D+%5B%22were%22%5D+%5B%22begging%22%5D+%5B%22the%22%5D+%5B%22Nazis%22%5D+%5B%22to%22%5D+%5B%22allow%22%5D+%5B%22them%22%5D+%5B%22to%22%5D+%5B%22escort%22%5D+%5B%22their%22%5D+%5B%22children%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'some of the mothers were begging the Nazis to allow them to escort their children . ', 'right': '', 'complete_match': 'some of the mothers were begging the Nazis to allow them to escort their children . ', 'testimony_id': 'usc_shoah_19111', 'shelfmark': ['USC Shoah Foundation 19111'], 'token_start': 11561, 'token_end': 11577}


In [25]:
fragment_3 = {}
fragment_3['original_sentence'] = "Leave the child with me, you know. And she was begging, you know, to leave this baby with her."
fragment_3['label']="Leave the child with me, you know. And she was begging, you know, to leave this baby with her."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Leave%22%5D+%5B%22the%22%5D+%5B%22child%22%5D+%5B%22with%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22she%22%5D+%5B%22was%22%5D+%5B%22begging%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22this%22%5D+%5B%22baby%22%5D+%5B%22with%22%5D+%5B%22her%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Leave the child with me , you know . And she was begging , you know , to leave this baby with her . ', 'right': '', 'complete_match': 'Leave the child with me , you know . And she was begging , you know , to leave this baby with her . ', 'testimony_id': 'usc_shoah_942', 'shelfmark': ['USC Shoah Foundation 942'], 'token_start': 10119, 'token_end': 10143}


In [26]:
fragment_4 = {}
fragment_4['original_sentence'] = " mothers and father kneeling on their hands and knees begging the Germans to leave their children alone."
fragment_4['label']= "(..) mothers and father kneeling on their hands and knees begging the Germans to leave their children alone."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22mothers%22%5D+%5B%22and%22%5D+%5B%22father%22%5D+%5B%22kneeling%22%5D+%5B%22on%22%5D+%5B%22their%22%5D+%5B%22hands%22%5D+%5B%22and%22%5D+%5B%22knees%22%5D+%5B%22begging%22%5D+%5B%22the%22%5D+%5B%22Germans%22%5D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22their%22%5D+%5B%22children%22%5D+%5B%22alone%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'mothers and father kneeling on their hands and knees begging the Germans to leave their children alone . ', 'right': '', 'complete_match': 'mothers and father kneeling on their hands and knees begging the Germans to leave their children alone . ', 'testimony_id': 'irn504520', 'shelfmark': ['USHMM RG-50.002*0015'], 'token_start': 5789, 'token_end': 5807}


In [27]:
fragment_5 = {}
fragment_5['original_sentence'] = "His struggle was like survival, but not for him, but for the children. He virtually got out and begged. He begged for my children he begged, you know."
fragment_5['label']= "His struggle was like survival, but not for him, but for the children. (..) He begged for my children he begged (..)."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22His%22%5D+%5B%22struggle%22%5D+%5B%22was%22%5D+%5B%22like%22%5D+%5B%22survival%22%5D+%5B%5D+%5B%22but%22%5D+%5B%22not%22%5D+%5B%22for%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22but%22%5D+%5B%22for%22%5D+%5B%22the%22%5D+%5B%22children%22%5D+%5B%5D+%5B%22He%22%5D+%5B%22virtually%22%5D+%5B%22got%22%5D+%5B%22out%22%5D+%5B%22and%22%5D+%5B%22begged%22%5D+%5B%5D+%5B%22He%22%5D+%5B%22begged%22%5D+%5B%22for%22%5D+%5B%22my%22%5D+%5B%22children%22%5D+%5B%22he%22%5D+%5B%22begged%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'His struggle was like survival , but not for him , but for the children . He virtually got out and begged . He begged for my children he begged , you know . ', 'right': '', 'complete_match': 'His struggle was like survival , but not for him , but for the children . He virtually got out and begged . He begged for my childr

In [28]:
add_testimonial_fragments(fragments)

### 2.  

In [29]:
lemmas = []

In [30]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

()


In [31]:
domain_term = "children"

In [32]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [33]:
fragment_1 = {}
fragment_1['original_sentence'] = "You saw little orphaned children in the street begging for food."
fragment_1['label']="You saw little orphaned children in the street begging for food."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22saw%22%5D+%5B%22little%22%5D+%5B%22orphaned%22%5D+%5B%22children%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22street%22%5D+%5B%22begging%22%5D+%5B%22for%22%5D+%5B%22food%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You saw little orphaned children in the street begging for food . ', 'right': '', 'complete_match': 'You saw little orphaned children in the street begging for food . ', 'testimony_id': 'irn504908', 'shelfmark': ['USHMM RG-50.030*0435'], 'token_start': 14468, 'token_end': 14480}


In [34]:
fragment_2 = {}
fragment_2['original_sentence'] = "I remember seeing children dying and begging for food."
fragment_2['label']="I remember seeing children dying and begging for food."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22remember%22%5D+%5B%22seeing%22%5D+%5B%22children%22%5D+%5B%22dying%22%5D+%5B%22and%22%5D+%5B%22begging%22%5D+%5B%22for%22%5D+%5B%22food%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I remember seeing children dying and begging for food . ', 'right': '', 'complete_match': 'I remember seeing children dying and begging for food . ', 'testimony_id': 'irn509226', 'shelfmark': ['USHMM RG-50.233*0144'], 'token_start': 3641, 'token_end': 3651}


In [35]:
fragment_3 = {}
fragment_3['original_sentence'] = "the little children sometimes were walking here and there, maybe begging or something,"
fragment_3['label']="(..) the little children sometimes were walking here and there, maybe begging or something (..)"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22little%22%5D+%5B%22children%22%5D+%5B%22sometimes%22%5D+%5B%22were%22%5D+%5B%22walking%22%5D+%5B%22here%22%5D+%5B%22and%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22maybe%22%5D+%5B%22begging%22%5D+%5B%22or%22%5D+%5B%22something%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the little children sometimes were walking here and there , maybe begging or something , ', 'right': '', 'complete_match': 'the little children sometimes were walking here and there , maybe begging or something , ', 'testimony_id': 'irn87783', 'shelfmark': ['USHMM RG-50.030*0770'], 'token_start': 24654, 'token_end': 24669}


In [36]:
fragment_4 = {}
fragment_4['original_sentence'] = "And the children-- it was terrible-- just to look on them, you know, walking around, begging for food"
fragment_4['label']= "And the children (..) just to look on them, you know, walking around, begging for food(..)."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22the%22%5D+%5B%22children%22%5D+%5B%5D%7B0%2C3%7D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22terrible%22%5D+%5B%5D%7B0%2C3%7D+%5B%22just%22%5D+%5B%22to%22%5D+%5B%22look%22%5D+%5B%22on%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22walking%22%5D+%5B%22around%22%5D+%5B%5D+%5B%22begging%22%5D+%5B%22for%22%5D+%5B%22food%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And the children -- it was terrible -- just to look on them , you know , walking around , begging for food ', 'right': '', 'complete_match': 'And the children -- it was terrible -- just to look on them , you know , walking around , begging for food ', 'testimony_id': 'usc_shoah_11296', 'shelfmark': ['USC Shoah Foundation 11296'], 'token_start': 5615, 'token_end': 5638}


In [37]:
fragment_5 = {}
fragment_5['original_sentence'] = "There were other children begging, going around yards, singing songs and begging."
fragment_5['label']= "There were other children begging, going around yards, singing songs and begging."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22There%22%5D+%5B%22were%22%5D+%5B%22other%22%5D+%5B%22children%22%5D+%5B%22begging%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22around%22%5D+%5B%22yards%22%5D+%5B%5D+%5B%22singing%22%5D+%5B%22songs%22%5D+%5B%22and%22%5D+%5B%22begging%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'There were other children begging , going around yards , singing songs and begging . ', 'right': '', 'complete_match': 'There were other children begging , going around yards , singing songs and begging . ', 'testimony_id': 'usc_shoah_23438', 'shelfmark': ['USC Shoah Foundation 23438'], 'token_start': 7905, 'token_end': 7920}


In [38]:
add_testimonial_fragments(fragments)

### 3.  

In [39]:
lemmas = []

In [40]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

()


In [41]:
domain_term = "leave him"

In [42]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [43]:
fragment_1 = {}
fragment_1['original_sentence'] = " And it was then that my mother started pleading and crying, saying he is only a little boy, he hasn’t done anything, and please leave him alone, you know – you know, just begging him."
fragment_1['label']=" And it was then that my mother started pleading and crying, saying he is only a little boy, he hasn’t done anything, and please leave him alone(..)"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22then%22%5D+%5B%22that%22%5D+%5B%22my%22%5D+%5B%22mother%22%5D+%5B%22started%22%5D+%5B%22pleading%22%5D+%5B%22and%22%5D+%5B%22crying%22%5D+%5B%5D+%5B%22saying%22%5D+%5B%22he%22%5D+%5B%22is%22%5D+%5B%22only%22%5D+%5B%22a%22%5D+%5B%22little%22%5D+%5B%22boy%22%5D+%5B%5D+%5B%22he%22%5D+%5B%22has%22%5D+%5B%5D+%5B%22done%22%5D+%5B%22anything%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22please%22%5D+%5B%22leave%22%5D+%5B%22him%22%5D+%5B%22alone%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D%7B0%2C3%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22just%22%5D+%5B%22begging%22%5D+%5B%22him%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And it was then that my mother started pleading and crying , saying he is only a little boy , he has n’t done anything , and please leave him alone , you know – you know , just begging him . ', 'rig

In [44]:
fragment_2 = {}
fragment_2['original_sentence'] = "My mother had to kneel down and beg them to leave me alone."
fragment_2['label']="My mother had to kneel down and beg them to leave me alone."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22My%22%5D+%5B%22mother%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22kneel%22%5D+%5B%22down%22%5D+%5B%22and%22%5D+%5B%22beg%22%5D+%5B%22them%22%5D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22me%22%5D+%5B%22alone%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'My mother had to kneel down and beg them to leave me alone . ', 'right': '', 'complete_match': 'My mother had to kneel down and beg them to leave me alone . ', 'testimony_id': 'irn504820', 'shelfmark': ['USHMM RG-50.030*0326'], 'token_start': 22905, 'token_end': 22919}


In [45]:
fragment_3 = {}
fragment_3['original_sentence'] = "We listened to the neighbors as she was begging them,"
fragment_3['label']="We listened to the neighbors as she was begging them, “Why? Just leave me, let me live!” "
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22listened%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22neighbors%22%5D+%5B%22as%22%5D+%5B%22she%22%5D+%5B%22was%22%5D+%5B%22begging%22%5D+%5B%22them%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We listened to the neighbors as she was begging them , ', 'right': '', 'complete_match': 'We listened to the neighbors as she was begging them , ', 'testimony_id': 'irn510468', 'shelfmark': ['USHMM RG-50.322*0014'], 'token_start': 13751, 'token_end': 13762}


In [46]:
fragment_4 = {}
fragment_4['original_sentence'] = "And she begged him to-- to-- to leave her son. And oh, that was so tragic there."
fragment_4['label']= "And she begged him to-- to-- to leave her son. And oh, that was so tragic there."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22she%22%5D+%5B%22begged%22%5D+%5B%22him%22%5D+%5B%22to%22%5D+%5B%5D%7B0%2C3%7D+%5B%22to%22%5D+%5B%5D%7B0%2C3%7D+%5B%22to%22%5D+%5B%22leave%22%5D+%5B%22her%22%5D+%5B%22son%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22oh%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22tragic%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And she begged him to -- to -- to leave her son . And oh , that was so tragic there . ', 'right': '', 'complete_match': 'And she begged him to -- to -- to leave her son . And oh , that was so tragic there . ', 'testimony_id': 'usc_shoah_2533', 'shelfmark': ['USC Shoah Foundation 2533'], 'token_start': 13732, 'token_end': 13754}


In [47]:
add_testimonial_fragments(fragments)

### 4.  

In [48]:
lemmas = ["kill","beg"]

In [49]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="kill"][]{0,10}[lemma="beg"])|([lemma="beg"][]{0,10}[lemma="kill"])


In [50]:
domain_term = "kill me"

In [51]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [52]:
fragment_1 = {}
fragment_1['original_sentence'] = "She begged them kill me, kill me."
fragment_1['label']="She begged them kill me, kill me."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22She%22%5D+%5B%22begged%22%5D+%5B%22them%22%5D+%5B%22kill%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22kill%22%5D+%5B%22me%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'She begged them kill me , kill me . ', 'right': '', 'complete_match': 'She begged them kill me , kill me . ', 'testimony_id': 'irn509153', 'shelfmark': ['USHMM RG-50.233*0070'], 'token_start': 18647, 'token_end': 18656}


In [53]:
fragment_2 = {}
fragment_2['original_sentence'] = "What I just mentioned to you before, the guy who begged him, kill me."
fragment_2['label']="What I just mentioned to you before, the guy who begged him, kill me. "
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22What%22%5D+%5B%22I%22%5D+%5B%22just%22%5D+%5B%22mentioned%22%5D+%5B%22to%22%5D+%5B%22you%22%5D+%5B%22before%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22guy%22%5D+%5B%22who%22%5D+%5B%22begged%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22kill%22%5D+%5B%22me%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'What I just mentioned to you before , the guy who begged him , kill me . ', 'right': '', 'complete_match': 'What I just mentioned to you before , the guy who begged him , kill me . ', 'testimony_id': 'usc_shoah_1419', 'shelfmark': ['USC Shoah Foundation 1419'], 'token_start': 12416, 'token_end': 12433}


In [54]:
fragment_3 = {}
fragment_3['original_sentence'] = "It took three shots, because Raul was begging him to kill him. Three shots, and Raul to get killed."
fragment_3['label']="It took three shots, because Raul was begging him to kill him. Three shots, and Raul to get killed."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22took%22%5D+%5B%22three%22%5D+%5B%22shots%22%5D+%5B%5D+%5B%22because%22%5D+%5B%22Raul%22%5D+%5B%22was%22%5D+%5B%22begging%22%5D+%5B%22him%22%5D+%5B%22to%22%5D+%5B%22kill%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22Three%22%5D+%5B%22shots%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22Raul%22%5D+%5B%22to%22%5D+%5B%22get%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'It took three shots , because Raul was begging him to kill him . Three shots , and Raul to get killed . ', 'right': '', 'complete_match': 'It took three shots , because Raul was begging him to kill him . Three shots , and Raul to get killed . ', 'testimony_id': 'usc_shoah_20405', 'shelfmark': ['USC Shoah Foundation 20405'], 'token_start': 21199, 'token_end': 21222}


In [55]:
fragment_4 = {}
fragment_4['original_sentence'] = "He was begging them, kill me, kill me, and no one killed him."
fragment_4['label']= "He was begging them, kill me, kill me, and no one killed him."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22He%22%5D+%5B%22was%22%5D+%5B%22begging%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22kill%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22kill%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22no%22%5D+%5B%22one%22%5D+%5B%22killed%22%5D+%5B%22him%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'He was begging them , kill me , kill me , and no one killed him . ', 'right': '', 'complete_match': 'He was begging them , kill me , kill me , and no one killed him . ', 'testimony_id': 'usc_shoah_4919', 'shelfmark': ['USC Shoah Foundation 4919'], 'token_start': 13690, 'token_end': 13707}


In [56]:
fragment_5 = {}
fragment_5['original_sentence'] = "And he begged that they kill him. And they shot him."
fragment_5['label']= "And he begged that they kill him. And they shot him."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22he%22%5D+%5B%22begged%22%5D+%5B%22that%22%5D+%5B%22they%22%5D+%5B%22kill%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22they%22%5D+%5B%22shot%22%5D+%5B%22him%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And he begged that they kill him . And they shot him . ', 'right': '', 'complete_match': 'And he begged that they kill him . And they shot him . ', 'testimony_id': 'HVT-136', 'shelfmark': ['Fortunoff Archive HVT-136'], 'token_start': 5773, 'token_end': 5786}


In [57]:
add_testimonial_fragments(fragments)

### 5.  

In [58]:
lemmas = []

In [59]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

()


In [60]:
domain_term = "don't kill me"

In [61]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [62]:
fragment_1 = {}
fragment_1['original_sentence'] = "Don't kill me. And I'm begging for my life."
fragment_1['label']="Don't kill me. And I'm begging for my life."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Do%22%5D+%5B%5D+%5B%22kill%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22I%22%5D+%5B%22%27m%22%5D+%5B%22begging%22%5D+%5B%22for%22%5D+%5B%22my%22%5D+%5B%22life%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "Do n't kill me . And I 'm begging for my life . ", 'right': '', 'complete_match': "Do n't kill me . And I 'm begging for my life . ", 'testimony_id': 'usc_shoah_7455', 'shelfmark': ['USC Shoah Foundation 7455'], 'token_start': 42673, 'token_end': 42686}


In [63]:
fragment_2 = {}
fragment_2['original_sentence'] = "As I told him, he's killed. I begged him not to shoot at to me. "
fragment_2['label']="As I told him, he's killed. I begged him not to shoot at to me. "
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22As%22%5D+%5B%22I%22%5D+%5B%22told%22%5D+%5B%22him%22%5D+%5B%5D+%5B%22he%22%5D+%5B%5D+%5B%22killed%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22begged%22%5D+%5B%22him%22%5D+%5B%22not%22%5D+%5B%22to%22%5D+%5B%22shoot%22%5D+%5B%22at%22%5D+%5B%22to%22%5D+%5B%22me%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "As I told him , he 's killed . I begged him not to shoot at to me . ", 'right': '', 'complete_match': "As I told him , he 's killed . I begged him not to shoot at to me . ", 'testimony_id': 'usc_shoah_1873', 'shelfmark': ['USC Shoah Foundation 1873'], 'token_start': 29247, 'token_end': 29266}


In [64]:
fragment_3 = {}
fragment_3['original_sentence'] = "And we begged them, please, don't shoot."
fragment_3['label']=" And we begged them, please, don't shoot."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22begged%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22please%22%5D+%5B%5D+%5B%22do%22%5D+%5B%5D+%5B%22shoot%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And we begged them , please , do n't shoot . ", 'right': '', 'complete_match': "And we begged them , please , do n't shoot . ", 'testimony_id': 'usc_shoah_7238', 'shelfmark': ['USC Shoah Foundation 7238'], 'token_start': 10708, 'token_end': 10719}


In [65]:
add_testimonial_fragments(fragments)

### 6.  

In [66]:
lemmas = ["beg","water"]

In [67]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="beg"][]{0,10}[lemma="water"])|([lemma="water"][]{0,10}[lemma="beg"])


In [68]:
domain_term = "water"

In [69]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [70]:
fragment_1 = {}
fragment_1['original_sentence'] = "to go from villager to villager begging for a little bit of water to fill the canteen."
fragment_1['label']="(..) to go from villager to villager begging for a little bit of water to fill the canteen."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22to%22%5D+%5B%22go%22%5D+%5B%22from%22%5D+%5B%22villager%22%5D+%5B%22to%22%5D+%5B%22villager%22%5D+%5B%22begging%22%5D+%5B%22for%22%5D+%5B%22a%22%5D+%5B%22little%22%5D+%5B%22bit%22%5D+%5B%22of%22%5D+%5B%22water%22%5D+%5B%22to%22%5D+%5B%22fill%22%5D+%5B%22the%22%5D+%5B%22canteen%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'to go from villager to villager begging for a little bit of water to fill the canteen . ', 'right': '', 'complete_match': 'to go from villager to villager begging for a little bit of water to fill the canteen . ', 'testimony_id': 'irn42018', 'shelfmark': ['USHMM RG-50.030*0586'], 'token_start': 27042, 'token_end': 27060}


In [71]:
fragment_2 = {}
fragment_2['original_sentence'] = "We begged a little waters from the SS. Nobody was hearing us."
fragment_2['label']="We begged a little waters from the SS. Nobody was hearing us."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22begged%22%5D+%5B%22a%22%5D+%5B%22little%22%5D+%5B%22waters%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22SS%22%5D+%5B%5D+%5B%22Nobody%22%5D+%5B%22was%22%5D+%5B%22hearing%22%5D+%5B%22us%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We begged a little waters from the SS . Nobody was hearing us . ', 'right': '', 'complete_match': 'We begged a little waters from the SS . Nobody was hearing us . ', 'testimony_id': 'irn504715', 'shelfmark': ['USHMM RG-50.030*0224'], 'token_start': 5395, 'token_end': 5409}


In [72]:
fragment_3 = {}
fragment_3['original_sentence'] = "You could hear them yell, scream, and beg for water, beg for, for help."
fragment_3['label']="You could hear them yell, scream, and beg for water, beg for, for help."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22could%22%5D+%5B%22hear%22%5D+%5B%22them%22%5D+%5B%22yell%22%5D+%5B%5D+%5B%22scream%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22beg%22%5D+%5B%22for%22%5D+%5B%22water%22%5D+%5B%5D+%5B%22beg%22%5D+%5B%22for%22%5D+%5B%5D+%5B%22for%22%5D+%5B%22help%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You could hear them yell , scream , and beg for water , beg for , for help . ', 'right': '', 'complete_match': 'You could hear them yell , scream , and beg for water , beg for , for help . ', 'testimony_id': 'irn504722', 'shelfmark': ['USHMM RG-50.030*0240'], 'token_start': 19575, 'token_end': 19594}


In [73]:
fragment_4 = {}
fragment_4['original_sentence'] = "The only thing is, we were so thirsty-- we weren't hungry-- we begged for water. We never got water."
fragment_4['label']= "The only thing is, we were so thirsty-- we weren't hungry-- we begged for water. We never got water."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22only%22%5D+%5B%22thing%22%5D+%5B%22is%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22thirsty%22%5D+%5B%5D%7B0%2C3%7D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%5D+%5B%22hungry%22%5D+%5B%5D%7B0%2C3%7D+%5B%22we%22%5D+%5B%22begged%22%5D+%5B%22for%22%5D+%5B%22water%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22never%22%5D+%5B%22got%22%5D+%5B%22water%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "The only thing is , we were so thirsty -- we were n't hungry -- we begged for water . We never got water . ", 'right': '', 'complete_match': "The only thing is , we were so thirsty -- we were n't hungry -- we begged for water . We never got water . ", 'testimony_id': 'usc_shoah_26585', 'shelfmark': ['USC Shoah Foundation 26585'], 'token_start': 20593, 'token_end': 20618}


In [74]:
fragment_5 = {}
fragment_5['original_sentence'] = "the begging of the sick people for medication or water-- that was unbearable."
fragment_5['label']= "the begging of the sick people for medication or water-- that was unbearable."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22begging%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22sick%22%5D+%5B%22people%22%5D+%5B%22for%22%5D+%5B%22medication%22%5D+%5B%22or%22%5D+%5B%22water%22%5D+%5B%5D%7B0%2C3%7D+%5B%22that%22%5D+%5B%22was%22%5D+%5B%22unbearable%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the begging of the sick people for medication or water -- that was unbearable . ', 'right': '', 'complete_match': 'the begging of the sick people for medication or water -- that was unbearable . ', 'testimony_id': 'usc_shoah_4753', 'shelfmark': ['USC Shoah Foundation 4753'], 'token_start': 5396, 'token_end': 5411}


In [75]:
add_testimonial_fragments(fragments)

### 7.  

In [76]:
lemmas = ["bread","beg"]

In [77]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="bread"][]{0,50}[lemma="beg"])|([lemma="beg"][]{0,50}[lemma="bread"])


In [78]:
domain_term = "bread"

In [79]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [80]:
fragment_1 = {}
fragment_1['original_sentence'] = "I feel like it's given to me because I was begging bread. I was begging from the farmers"
fragment_1['label']="I feel like it's given to me because I was begging bread. I was begging from the farmers (..)."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22feel%22%5D+%5B%22like%22%5D+%5B%22it%22%5D+%5B%5D+%5B%22given%22%5D+%5B%22to%22%5D+%5B%22me%22%5D+%5B%22because%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22begging%22%5D+%5B%22bread%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22begging%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22farmers%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I feel like it 's given to me because I was begging bread . I was begging from the farmers ", 'right': '', 'complete_match': "I feel like it 's given to me because I was begging bread . I was begging from the farmers ", 'testimony_id': 'usc_shoah_24615', 'shelfmark': ['USC Shoah Foundation 24615'], 'token_start': 27433, 'token_end': 27453}


In [81]:
fragment_2 = {}
fragment_2['original_sentence'] = "We went into some houses, and we begged for bread, for some things."
fragment_2['label']="We went into some houses, and we begged for bread, for some things."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22went%22%5D+%5B%22into%22%5D+%5B%22some%22%5D+%5B%22houses%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22we%22%5D+%5B%22begged%22%5D+%5B%22for%22%5D+%5B%22bread%22%5D+%5B%5D+%5B%22for%22%5D+%5B%22some%22%5D+%5B%22things%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We went into some houses , and we begged for bread , for some things . ', 'right': '', 'complete_match': 'We went into some houses , and we begged for bread , for some things . ', 'testimony_id': 'usc_shoah_3952', 'shelfmark': ['USC Shoah Foundation 3952'], 'token_start': 23792, 'token_end': 23808}


In [82]:
fragment_3 = {}
fragment_3['original_sentence'] = "And then I would come to a village and beg for bread, beg for food, beg for a place to stay."
fragment_3['label']="And then I would come to a village and beg for bread, beg for food, beg for a place to stay."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22then%22%5D+%5B%22I%22%5D+%5B%22would%22%5D+%5B%22come%22%5D+%5B%22to%22%5D+%5B%22a%22%5D+%5B%22village%22%5D+%5B%22and%22%5D+%5B%22beg%22%5D+%5B%22for%22%5D+%5B%22bread%22%5D+%5B%5D+%5B%22beg%22%5D+%5B%22for%22%5D+%5B%22food%22%5D+%5B%5D+%5B%22beg%22%5D+%5B%22for%22%5D+%5B%22a%22%5D+%5B%22place%22%5D+%5B%22to%22%5D+%5B%22stay%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And then I would come to a village and beg for bread , beg for food , beg for a place to stay . ', 'right': '', 'complete_match': 'And then I would come to a village and beg for bread , beg for food , beg for a place to stay . ', 'testimony_id': 'usc_shoah_12455', 'shelfmark': ['USC Shoah Foundation 12455'], 'token_start': 2764, 'token_end': 2788}


In [83]:
fragment_4 = {}
fragment_4['original_sentence'] = "I used to go to the farmers and, actually, sometimes beg. Beg for a piece of bread."
fragment_4['label']= " I used to go to the farmers and, actually, sometimes beg. Beg for a piece of bread."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22used%22%5D+%5B%22to%22%5D+%5B%22go%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22farmers%22%5D+%5B%22and%22%5D+%5B%5D+%5B%22actually%22%5D+%5B%5D+%5B%22sometimes%22%5D+%5B%22beg%22%5D+%5B%5D+%5B%22Beg%22%5D+%5B%22for%22%5D+%5B%22a%22%5D+%5B%22piece%22%5D+%5B%22of%22%5D+%5B%22bread%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I used to go to the farmers and , actually , sometimes beg . Beg for a piece of bread . ', 'right': '', 'complete_match': 'I used to go to the farmers and , actually , sometimes beg . Beg for a piece of bread . ', 'testimony_id': 'usc_shoah_23572', 'shelfmark': ['USC Shoah Foundation 23572'], 'token_start': 14456, 'token_end': 14477}


In [84]:
fragment_5 = {}
fragment_5['original_sentence'] = "And I went to the civilians, the French or the other ones, and begged them for bread or other things."
fragment_5['label']= "And I went to the civilians, the French or the other ones, and begged them for bread or other things."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22went%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22civilians%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22French%22%5D+%5B%22or%22%5D+%5B%22the%22%5D+%5B%22other%22%5D+%5B%22ones%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22begged%22%5D+%5B%22them%22%5D+%5B%22for%22%5D+%5B%22bread%22%5D+%5B%22or%22%5D+%5B%22other%22%5D+%5B%22things%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And I went to the civilians , the French or the other ones , and begged them for bread or other things . ', 'right': '', 'complete_match': 'And I went to the civilians , the French or the other ones , and begged them for bread or other things . ', 'testimony_id': 'usc_shoah_6526', 'shelfmark': ['USC Shoah Foundation 6526'], 'token_start': 22842, 'token_end': 22865}


In [85]:
add_testimonial_fragments(fragments)